Magic or special oe dunder methods start and end with a double underscore ( dunder ) and can use objects behave like built-ins such as numbers, lists, dict. <br>
Common used operator overloading are __init__, __add__, __repr__ <br>

Initialization of new objects <br>
Object representation <br>
Enable iteration <br>
Operator overloading (addition) <br>
Method invocation <br>
Context manager support (with statement) <br>


In [7]:
#Common magic methods or operator overloading

d = { 'one' : 1, 'two': 2 }

print( d['two'])
print( d.__getitem__('two'))

2
2


In [8]:
dict1 = {1 : "ABC"}
dict2 = {2 : "EFG"}

dict1 + dict2 #ERROR:  because the dictionary type doesn't support addition

TypeError: unsupported operand type(s) for +: 'dict' and 'dict'

In [20]:
class addDict(dict):
    def __add__(self, otherobj):
        self.update(otherobj)
        return addDict(self)
    
d1 = addDict({ 1: 'ABC'})
d2 = addDict({  2: 'XYZ' })

d1 + d2 

{1: 'ABC', 2: 'XYZ'}

In [35]:
from functools import total_ordering


@total_ordering
class Account:
    'A simple account class'

    def __init__(self, owner, amount=0):
        'This is the constructor that lets us create objects from this class'
        self.owner = owner
        self.amount = amount
        self._transactions = []

    def __repr__(self):
        return 'Account({!r}, {!r})'.format(self.owner, self.amount)

    def __str__(self):
        return 'Account of {} with starting amount: {}'.format(self.owner,
                                                               self.amount)

    def add_transaction(self, amount):
        if not isinstance(amount, int):
            raise ValueError('please use int for amount')
        self._transactions.append(amount)
        

    @property
    def balance(self):
        return self.amount + sum(self._transactions)

    def __len__(self):
        return len(self._transactions)

    def __getitem__(self, position):
        return self._transactions[position]

    def __reversed__(self):
        return self[::-1]

    def __eq__(self, other):
        return self.balance == other.balance

    def __lt__(self, other):
        return self.balance < other.balance

    def __add__(self, other):
        owner = '{}&{}'.format(self.owner, other.owner)
        start_amount = self.amount + other.amount
        acc = Account(owner, start_amount)
        for t in list(self) + list(other):
            acc.add_transaction(t)
        return acc

    def __call__(self):
        print('Start amount: {}'.format(self.amount))
        print('Transactions: ')
        for transaction in self:
            print(transaction)
        print('\nBalance: {}'.format(self.balance))

    def __enter__(self):
        print('ENTER WITH: making backup of transactions for rollback')
        self._copy_transactions = list(self._transactions)
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        print('EXIT WITH:', end=' ')
        if exc_type:
            self._transactions = self._copy_transactions
            print('rolling back to previous transactions')
            print('transaction resulted in {} ({})'.format(exc_type.__name__, exc_val))  # noqa E501
        else:
            print('transaction ok')


##main
acc = Account('bob', 10)

#object representation
print( str(acc))
print(repr(acc))

acc.add_transaction(20)

print(acc.balance)

#Iteration: __len__, __getitem__
print(len(acc))

#calling python objects: __call__
print(acc())

# Context Manager support: __enter__ and __exit__





Account of bob with starting amount: 10
Account('bob', 10)
30
1
Start amount: 10
Transactions: 
20

Balance: 30
None


**Context Manager**: used for allocation and releasing of resources.<br>
And, Any pair of operations that need to be performed before or after a procedure like try/except/finally. <br>
When to use: when the following patterns supports.<br>
open-close, Locl-Release, Change-Reset, Enter-Exit, Start-Stop.



In [38]:
def kill_process(pid):
    try:
        os.kill( pid, signal.SIGKILL )
    except ProcessLookupError:
        pass
    
##Replace to

from contextlib import suppress

def kill_process(pid):
    with suppress(ProcessLookupError):
        os.kill(pid, signal.SIGKILL)

In [41]:
#Method 1
class CManager(object):
    def __init__(self):
        print('__init__')
        
    def __enter__(self):
        print("__enter__")
        return self
    
    def __exit__(self, type, value, traceback ):
        print('__exit__', type,value)
        return True
    
    def __del__(self):
        print('__del__', self)
        
        
with CManager() as c:
    print("doing something with c:", c)
    raise RuntimeError()
    print("finished doing something")
    

__init__
__enter__
__del__ <__main__.CManager object at 0x00000151B7C702E8>
doing something with c: <__main__.CManager object at 0x00000151B7C70400>
__exit__ <class 'RuntimeError'> 


In [42]:
#Method 2: simple context managers can also be written using generators and contextmanager decorator

from contextlib import contextmanager

@contextmanager
def context_manager_func():
    setup()

    # yield must be wrapped in try/finally
    # to trap any exceptions raised in the calling code
    try:
        yield
    finally:
        teardown()

Advantages: Context managers give us a reliable method to clean up resources. Usaully, __del__ called when reference count is zero.

In [46]:
#Make sure an open stream is closed at the end
class File():

    def __init__(self, filename, mode):
        self.filename = filename
        self.mode = mode

    def __enter__(self):
        self.open_file = open(self.filename, self.mode)
        return self.open_file

    def __exit__(self, *args):
        self.open_file.close()

files = []
for _ in range(10000):
    with File('foo.txt', 'w+') as infile:
        infile.write('foo')
        files.append(infile)

In [48]:
#Synchronising access to shared resources

import fcntl
import os

class PidFile(object):
    """Context manager that locks a pid file.  Implemented as class
    not generator because daemon.py is calling .__exit__() with no parameters
    instead of the None, None, None specified by PEP-343."""
    # pylint: disable=R0903

    def __init__(self, path):
        self.path = path
        self.pidfile = None

    def __enter__(self):
        self.pidfile = open(self.path, "a+")
        try:
            fcntl.flock(self.pidfile.fileno(), fcntl.LOCK_EX | fcntl.LOCK_NB)
        except IOError:
            raise SystemExit("Already running according to " + self.path)
        self.pidfile.seek(0)
        self.pidfile.truncate()
        self.pidfile.write(str(os.getpid()))
        self.pidfile.flush()
        self.pidfile.seek(0)
        return self.pidfile

    def __exit__(self, exc_type=None, exc_value=None, exc_tb=None):
        try:
            self.pidfile.close()
        except IOError as err:
            # ok if file was just closed elsewhere
            if err.errno != 9:
                raise
        os.remove(self.path)

# example usage
import daemon
context = daemon.DaemonContext()
context.pidfile = PidFile("/var/run/mydaemon")

ModuleNotFoundError: No module named 'fcntl'

In [56]:
#Timing the execution of a code block
import time
import http

class Timer:
    def __init__(self, name):
        self.name = name

    def __enter__(self):
        self.start = time.time()

    def __exit__(self, *args):
        self.end = time.time()
        self.interval = self.end - self.start
        print("%s took: %0.3f seconds" % (self.name, self.interval))
        return False
    
with Timer('fetching google homepage'):
    conn = http.client.HTTPConnection('google.com')
    conn.request('GET', '/')

fetching google homepage took: 0.099 seconds


In [ ]:
#Managing a pool of processes
from multiprocessing import Pool

def f(x):
    return x*x

with Pool(processes=4) as pool:  # start 4 worker processes

    # evaluate "f(10)" asynchronously in a single process
    result = pool.apply_async(f, (10,)) 
    #print(result.get(timeout=3)) 
    # prints "100"

    print(pool.map(f, range(10))) 

In [ ]:
#Error handling patterns;

try:
    do_stuff()
except UnicodeError:
    print("annoying")
except ValueError:
    print("hrmm")
except OSError:
    print("Alert system admins")
except:
    print("other")

In [ ]:
class my_error_handling(object):
    def __enter__(self):
        pass
    def __exit__(self, exc_type, exc_val, exc_tb ):
        if issubclass( exc_type, UnicodeError ):
            print("Annoying")
        elif issubclass( exc_type, ValueError ):
            print("Hrmm")
            
with my_error_handling():
    do_stuff()